# Distortion Classify

In [3]:
import torch
import torchaudio
import os
import pandas as pd
import csv
import numpy as np
import itertools
from src.effectapplier import EffectApplier
from src.util import play_audio

### Data Pre-processing

In [2]:
from src.util import fade_in, right_pad

_SAMPLE_DIR = "_assets/DATASET/IDMT-SMT-AUDIO-EFFECTS"
_OUT_DIR = "_assets/DATASET/GT-FX-DRY"

if not os.path.exists('%s' % _OUT_DIR):
    os.makedirs('%s' % _OUT_DIR)

file_index = 0

for file in os.listdir(_SAMPLE_DIR):
    waveform, sr = torchaudio.load(os.path.join(_SAMPLE_DIR, file))

    waveform = fade_in(waveform, sr)
    waveform = right_pad(waveform, sr, length=3)

    if file_index < 10:
        ind = '000' +str(file_index)
    elif file_index < 100:
        ind = '00' +str(file_index)
    elif file_index < 1000:
        ind = '0' +str(file_index)
    else:
        ind = file_index

    filename = "C50-%s.wav" % ind
    output_path = os.path.join(_OUT_DIR, filename)
    
    waveform = waveform / abs(waveform).max()
    torchaudio.save(output_path, waveform, sr, encoding="PCM_S", bits_per_sample=16, format="wav")
    
    file_index += 1

In [2]:
_SAMPLE_DIR = "_assets/DATASET/GT-FX-DRY"
label_list = ["filename", "distortion", "chorus", "tremolo", "delay", "reverb",
              "distortion_class", "chorus_class", "tremolo_class", "delay_class", "reverb_class",
              "distortion_value", "chorus_value", "tremolo_value", "delay_value", "reverb_value"]
fx_list = [0,1,2,3,4]
parameters = [0.2, 0.5, 0.8]

### Apply effects to preprocessed data

In [30]:
_OUT_DIR = "_assets/DATASET/GT-FX-C53"

if not os.path.exists('%s' % _OUT_DIR):
    os.makedirs('%s' % _OUT_DIR)

# writing the data into the file
CSV_PATH = _OUT_DIR + "/annotation.csv"
file = open(CSV_PATH, mode='a', newline ='')

with file:
    write = csv.writer(file)
    write.writerow(label_list)
    for file in os.listdir(_SAMPLE_DIR):
            file_index = file[4:-4]
            for nEffects in range(1, 4):
                filename_o = "C53-%s-%s-" % (file_index, nEffects)

                p = list(itertools.combinations(fx_list, nEffects))
                val_list = list(itertools.product(parameters, repeat=nEffects))
                
                for effects in p:
                    for val in val_list:
                        eff = EffectApplier(_SAMPLE_DIR, file, _OUT_DIR, filename_o)
                        for index, fx in enumerate(effects):
                            eff.addEffect(fx, val[index], scale=0.33)
                        write.writerow(eff.save())

for file in os.listdir(_OUT_DIR):
    if file.startswith('C53-0000-3'):
        fx = file[-9:-4]
        fx = [*fx]

        if '1' not in fx and '3' not in fx:
            path = os.path.join(_OUT_DIR, file)
            print(file)
            waveform, sample_rate = torchaudio.load(path)
            play_audio(waveform, sample_rate)


C53-0000-3-00111-00222.wav


C53-0000-3-01011-02022.wav


C53-0000-3-01101-02202.wav


C53-0000-3-01110-02220.wav


C53-0000-3-10011-20022.wav


C53-0000-3-10101-20202.wav


C53-0000-3-10110-20220.wav


C53-0000-3-11001-22002.wav


C53-0000-3-11010-22020.wav


C53-0000-3-11100-22200.wav
